In [40]:
from pyhive import hive
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
from sqlalchemy import create_engine

In [41]:
conn = hive.Connection(host='localhost', port=10000, username='rodrigo', database='youtube')
engine = create_engine('hive://', creator=lambda: conn)

Could not connect to any of [('127.0.0.1', 10000)]


TTransportException: Could not connect to any of [('127.0.0.1', 10000)]

In [ ]:
tipos = {
    'data_extracao':  'datetime64[ns]',
    'total_tags': 'int32',
    'total_visualizacoes_turno': 'int32',
    'total_comentarios_turno' :  'int32',
    'total_likes_turno' : 'float',
    'total_caractere_video': 'int32',
    'turno_extracao': 'string'

}

In [ ]:
df_result = pd.read_sql_query(
    ' SELECT data_extracao, '
    ' total_caractere_video, '
    ' total_tags, '
    ' total_visualizacoes_turno,  '
    ' total_comentarios_turno,	 '
    ' total_likes_turno, '
    ' turno_extracao '
    ' FROM TOTAL_VISUALIZACOES_POR_SEMANA '
    ' where TOTAL_VISUALIZACOES_POR_SEMANA.assunto = "assunto_cities_skylines" '
    ' AND TOTAL_VISUALIZACOES_POR_SEMANA.data_extracao = "2023-10-27"', 
    con=engine,
    dtype=tipos
)

# Comparação de views/likes/comentários em relação ao dia anterior

- views

In [ ]:
tipos = {
    'data_extracao':  'datetime64[ns]',
    'total_visualizacoes_turno': 'int32',
    'turno_extracao': 'string',

}

In [ ]:
df_result = pd.read_sql_query(
    ' SELECT data_extracao, '
    ' turno_extracao, '
    ' total_visualizacoes_turno '
    ' FROM TOTAL_VISUALIZACOES_POR_SEMANA '
    ' where TOTAL_VISUALIZACOES_POR_SEMANA.assunto = "assunto_cities_skylines" '
    ' AND ID_CANAL="UCrOH1V-FyMunBIMrKL0y0xQ" '
    ' AND  data_extracao in ("2023-10-15", "2023-10-16")', 
    con=engine,
    dtype=tipos
)
df_result.head(20)

,data_extracao,turno_extracao,total_visualizacoes_turno


In [ ]:
date_range = pd.date_range(start='2023-10-15', periods=13, freq='D')
date_range

DatetimeIndex(['2023-10-15', '2023-10-16', '2023-10-17', '2023-10-18',
               '2023-10-19', '2023-10-20', '2023-10-21', '2023-10-22',
               '2023-10-23', '2023-10-24', '2023-10-25', '2023-10-26',
               '2023-10-27'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
df_percentual_variacao = df_result.groupby('data_extracao') \
    .agg(total_visualizacoes=('total_visualizacoes_turno', 'sum')).reset_index() 
df_percentual_variacao['total_visualizacoes_dia_anterior'] = df_percentual_variacao['total_visualizacoes'].shift(1)
# df_percentual_variacao['PERCENTUAL_VISUALIZACAO'] = (df_percentual_variacao['total_visualizacoes_dia_anterior'] / df_percentual_variacao['total_visualizacoes']) * 100 
df_percentual_variacao.dropna(0, inplace=True)
# df_percentual_variacao.drop('total_visualizacoes_dia_anterior', axis=1, inplace=True)
df_percentual_variacao.head(25)

,data_extracao,total_visualizacoes,total_visualizacoes_dia_anterior
0,2023-10-15,4562,0.0
1,2023-10-16,10751,4562.0


In [ ]:
df_percentual_variacao = df_percentual_variacao[df_percentual_variacao['data_extracao'] == '2023-10-16	']

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Indicator(
        mode='number+delta',
        value=df_percentual_variacao['total_visualizacoes'].iloc[0],
        delta={
            'reference':  round(int(df_percentual_variacao['total_visualizacoes_dia_anterior'].iloc[0]), 2) ,
            'relative': True,
            'position': 'bottom',
            'valueformat': '.2%',
            },
    )
)
fig.show()

In [ ]:
df_percentual_variacao.memory_usage(deep=True).sum() / (1024 ** 2)

0.000331878662109375

-  comentários

In [ ]:
tipos = {
    'data_extracao':  'datetime64[ns]',
    'total_comentarios_turno': 'int32',
    'turno_extracao': 'string',

}

In [ ]:
df_result = pd.read_sql_query(
    ' SELECT data_extracao, '
    ' turno_extracao, '
    ' total_comentarios_turno '
    ' FROM TOTAL_VISUALIZACOES_POR_SEMANA '
    ' where TOTAL_VISUALIZACOES_POR_SEMANA.assunto = "assunto_cities_skylines" '
    ' AND ID_CANAL="UCrOH1V-FyMunBIMrKL0y0xQ"', 
    con=engine,
    dtype=tipos
)
df_result.head()

,data_extracao,turno_extracao,total_comentarios_turno
0,2023-10-15,Noite,23
1,2023-10-15,Tarde,23
2,2023-10-16,Manhã,4
3,2023-10-16,Noite,1
4,2023-10-16,Tarde,1


In [ ]:
df_percentual_variacao = df_result.groupby('data_extracao') \
    .agg(total_comentarios=('total_comentarios_turno', 'sum')).reset_index() 
df_percentual_variacao['total_comentarios_dia_anterior'] = df_percentual_variacao['total_comentarios'].diff()
df_percentual_variacao['PERCENTUAL_COMENTARIOS'] = (df_percentual_variacao['total_comentarios_dia_anterior'] / df_percentual_variacao['total_comentarios']) * 100 
df_percentual_variacao.fillna(0, inplace=True)
df_percentual_variacao.drop('total_comentarios_dia_anterior', axis=1, inplace=True)
df_percentual_variacao.head(25)

,data_extracao,total_comentarios,PERCENTUAL_COMENTARIOS
0,2023-10-15,46,0.000000
1,2023-10-16,61,24.590164
2,2023-10-19,732,91.666667
3,2023-10-20,575,-27.304348
4,2023-10-21,560,-2.678571
5,2023-10-22,835,32.934132
6,2023-10-23,1729,51.706189
7,2023-10-24,1251,-38.209432
8,2023-10-25,1980,36.818182
9,2023-10-26,3761,47.354427


- Likes 

In [ ]:
tipos = {
    'data_extracao':  'datetime64[ns]',
    'total_likes_turno': 'int32',
    'turno_extracao': 'string',

}

In [ ]:
df_result = pd.read_sql_query(
    ' SELECT data_extracao, '
    ' turno_extracao, '
    ' total_likes_turno '
    ' FROM TOTAL_VISUALIZACOES_POR_SEMANA '
    ' where TOTAL_VISUALIZACOES_POR_SEMANA.assunto = "assunto_cities_skylines" '
    ' AND ID_CANAL="UCrOH1V-FyMunBIMrKL0y0xQ"', 
    con=engine,
    dtype=tipos
)
df_result.head()

,data_extracao,turno_extracao,total_likes_turno
0,2023-10-15,Noite,156
1,2023-10-15,Tarde,531
2,2023-10-16,Manhã,139
3,2023-10-16,Noite,31
4,2023-10-16,Tarde,66


In [ ]:
df_percentual_variacao = df_result.groupby('data_extracao') \
    .agg(total_likes=('total_likes_turno', 'sum')).reset_index() 
df_percentual_variacao['total_likes_dia_anterior'] = df_percentual_variacao['total_likes'].diff()
df_percentual_variacao['PERCENTUAL_LIKES'] = (df_percentual_variacao['total_likes_dia_anterior'] / df_percentual_variacao['total_likes']) * 100 
df_percentual_variacao.fillna(0, inplace=True)
df_percentual_variacao.drop('total_likes_dia_anterior', axis=1, inplace=True)
df_percentual_variacao.head(25)

,data_extracao,total_likes,PERCENTUAL_LIKES
0,2023-10-15,687,0.000000
1,2023-10-16,1753,60.810040
2,2023-10-19,10675,83.578454
3,2023-10-20,7585,-40.738299
4,2023-10-21,10044,24.482278
5,2023-10-22,10181,1.345644
6,2023-10-23,9333,-9.086039
7,2023-10-24,9334,0.010714
8,2023-10-25,9709,3.862396
9,2023-10-26,13994,30.620266


# Dia em que os vídeos são Públicados com alta frequência

In [ ]:
tipos = {
    'semana_traduzida': 'string',
    'nm_canal': 'string',
    'total_videos' : 'int32',
    'data_publicacao' : 'datetime64[ns]',
}

In [ ]:
df_publicacao_video = pd.read_sql_query(
    'SELECT semana_traduzida, '
    ' nm_canal,  '
    ' total_videos, '
    ' data_publicacao '
    ' FROM youtube.total_video_publicado_semana'
    ' WHERE assunto = "assunto_cities_skylines" ',

    dtype=tipos,
    con=engine
)

In [ ]:
df_publicacao_video.head()

,semana_traduzida,nm_canal,total_videos,data_publicacao
0,Domingo,Irmãos Cities,1,2023-10-15
1,Domingo,CANAL HAND - GAME WITH MUSIC,1,2023-10-15
2,Domingo,GAC TV,1,2023-10-15
3,Domingo,Lucas gameplays brasil,1,2023-10-15
4,Domingo,O Veio Games,1,2023-10-15


In [ ]:
def indice_semana(dia: str) -> int: 
    dias_semana = {
        'Domingo': 1,
        'Segunda-feira' : 2,
        'Terça-feira' : 3,
        'Quarta-feira' : 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado' : 7
    }

    return dias_semana.get(dia)


In [ ]:
df_publicacao_video.head()

,semana_traduzida,nm_canal,total_videos,data_publicacao
0,Domingo,Irmãos Cities,1,2023-10-15
1,Domingo,CANAL HAND - GAME WITH MUSIC,1,2023-10-15
2,Domingo,GAC TV,1,2023-10-15
3,Domingo,Lucas gameplays brasil,1,2023-10-15
4,Domingo,O Veio Games,1,2023-10-15


In [ ]:

# df_publicacao_video['indice_semana'] = df_publicacao_video['semana_traduzida'].apply(indice_semana)
# df_publicacao_video.sort_values(by='indice_semana', inplace=True)
df_publicacao_video = df_publicacao_video.groupby('semana_traduzida').sum('total_videos').reset_index()
df_publicacao_video['indice_semana'] = df_publicacao_video['semana_traduzida'].apply(indice_semana)
df_publicacao_video.sort_values(by='indice_semana', inplace=True)
df_publicacao_video.drop(['indice_semana'], axis=1, inplace=True)
df_publicacao_video.head(7)

,semana_traduzida,total_videos
0,Domingo,25
3,Segunda-feira,40
6,Terça-feira,39
1,Quarta-feira,37
2,Quinta-feira,39
4,Sexta-feira,34
5,Sábado,21


- Cor dia de publicação

In [ ]:
fig = px.bar(df_publicacao_video, x='semana_traduzida',y='total_videos', text_auto=True)
fig.update_layout(title_text='Envio de Vídeo por Semana', showlegend=True, title=dict(x=0.5) )

fig.update_layout(
    margin=dict(l=20, r=20, t=40, b=20, pad=4),
    paper_bgcolor='LightSteelBlue'
    
)

fig.show()

- Ajuste automático de Layout

In [ ]:
fig = px.bar(df_publicacao_video, x='semana_traduzida',y='total_videos', text_auto=True)


fig.update_layout(
    title_text='Envio de Vídeo por Semana',
    showlegend=True,
    title=dict(x=0.5, font=dict(color='white')),
    plot_bgcolor='#1F2326',
    yaxis=dict(visible=False),  
    margin=dict(l=20, r=20, t=40, b=20, pad=4),
    paper_bgcolor='#1F2326',
    xaxis=dict(title='', tickfont=dict(color='white')),
    legend=dict(font=dict(color='white')),
)
fig.update_traces(
    textfont_color='white',
    marker_color='#FFA500',
    textfont_size=16  
)

fig.show()

# Total Víews DIA por Vídeo

In [ ]:
tipo = {
    'turno_extracao': 'string',
    'titulo_video': 'string',
    'id_video': 'string',
    'total_visualizacoes_turno': 'int32',
    'indice_turno': 'int32'
}

In [ ]:
def indice_turno(turno: str):
    ds_turno = {
        'Manhã': '1',
        'Tarde' : '2',
        'Noite' : '3'
    }
    return ds_turno.get(turno)

In [ ]:
df_views_video = pd.read_sql_query(
    ' SELECT  '
    ' turno_extracao, '
    ' titulo_video, '
    ' id_video, '
    ' total_visualizacoes_turno , '
    ' CASE WHEN turno_extracao == "Manhã" Then 1 when turno_extracao == "Tarde" then 2 else 3 end as indice_turno  '
    ' FROM total_visualizacoes_por_semana tvps '
    ' where id_canal  = "UCrOH1V-FyMunBIMrKL0y0xQ" '
    ' AND data_extracao = "2023-10-21" '
    ' AND total_visualizacoes  > 0 ',
    con=engine,
    dtype=tipo
)
df_views_video.head() 

,turno_extracao,titulo_video,id_video,total_visualizacoes_turno,indice_turno
0,Manhã,MODDERS decidem fazer O MELHOR JOGO DE GERENCI...,XOUzWJ0bHuw,86,1
1,Noite,MODDERS decidem fazer O MELHOR JOGO DE GERENCI...,XOUzWJ0bHuw,32,3
2,Tarde,MODDERS decidem fazer O MELHOR JOGO DE GERENCI...,XOUzWJ0bHuw,33,2
3,Manhã,Planejamento inicial PODE SALVAR SUA CIDADE! 🚗...,dRzwiPwGH_M,274,1
4,Noite,Planejamento inicial PODE SALVAR SUA CIDADE! 🚗...,dRzwiPwGH_M,67,3


In [ ]:
df_views_video.shape

(40, 5)

In [ ]:
df_views_video.head()

,turno_extracao,titulo_video,id_video,total_visualizacoes_turno,indice_turno
0,Manhã,MODDERS decidem fazer O MELHOR JOGO DE GERENCI...,XOUzWJ0bHuw,86,1
1,Noite,MODDERS decidem fazer O MELHOR JOGO DE GERENCI...,XOUzWJ0bHuw,32,3
2,Tarde,MODDERS decidem fazer O MELHOR JOGO DE GERENCI...,XOUzWJ0bHuw,33,2
3,Manhã,Planejamento inicial PODE SALVAR SUA CIDADE! 🚗...,dRzwiPwGH_M,274,1
4,Noite,Planejamento inicial PODE SALVAR SUA CIDADE! 🚗...,dRzwiPwGH_M,67,3


In [ ]:
df_views_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   turno_extracao             40 non-null     string
 1   titulo_video               40 non-null     string
 2   id_video                   40 non-null     string
 3   total_visualizacoes_turno  40 non-null     int32 
 4   indice_turno               40 non-null     int64 
dtypes: int32(1), int64(1), string(3)
memory usage: 1.5 KB


In [ ]:
df_views_video = df_views_video.groupby(['id_video', 'titulo_video']).sum('total_visualizacoes_turno').reset_index()
df_views_video.head()

,id_video,titulo_video,total_visualizacoes_turno,indice_turno
0,XOUzWJ0bHuw,MODDERS decidem fazer O MELHOR JOGO DE GERENCI...,302,12
1,dRzwiPwGH_M,Planejamento inicial PODE SALVAR SUA CIDADE! 🚗...,880,12
2,totHMIv6_hI,PEQUENA FORTUNA EM SERVIÇOS PÚBLICOS BÁSICOS 🏙...,64184,12
3,up4KrHrRwLY,"DO CRIADOR DA MELHOR CIDADE JÁ FEITA, A NOVA M...",4476,12
4,vy-upTpchig,PRIMEIRA PRAÇA DA CIDADE SAI DO PAPEL 🏙️ - Cit...,49262,10


In [ ]:

df_views_video.sort_values(by='total_visualizacoes_turno', ascending=True, inplace=True)
fig = px.bar(df_views_video, x='total_visualizacoes_turno', y='id_video', orientation='h', text_auto=True)


fig.update_layout(
    title_text='Total Visualizações',
    showlegend=True,
    title=dict(x=0.5, font=dict(color='white')),
    plot_bgcolor='#1F2326',
    margin=dict(l=10, r=20, t=40, b=20, pad=2),
    paper_bgcolor='#1F2326',
    yaxis=dict(title='', tickfont=dict(color='white', size=16), tickmode='array', ticklen=1),
    xaxis=dict(title='', tickfont=dict(color='white'), visible=False),
    legend=dict(font=dict(color='white'), orientation='h', y=4)
)
    
fig.update_traces(
    textfont_color='white',
    marker_color='#04BE5F',
    textfont_size=18 ,
    textposition='outside',  
)


fig.show()


- Selecionando Vídeo - (visualização, vídeo, comentário, likes)

In [ ]:
tipos = {
    'data_extracao' : 'datetime64[ns]',
    'turno_extracao' : 'string',
    'total_visualizacoes_turno' : 'int'
}

- Desempenho Vídeo
    

In [ ]:
df_desempenho_video = pd.read_sql_query(
    ' SELECT data_extracao, '
    ' turno_extracao, '
    ' total_visualizacoes_turno,'
    ' CASE WHEN turno_extracao == "Manhã" Then 1 when turno_extracao == "Tarde" then 2 else 3 end as INDICE_TURNO  '
    ' FROM total_visualizacoes_por_semana tvps '
    ' where tvps.assunto  = "assunto_cities_skylines" '
    ' AND tvps.id_video = "wCLSZxLfUAk" ' ,
    con=engine,
    dtype=tipos
)
df_desempenho_video.head()

,data_extracao,turno_extracao,total_visualizacoes_turno,indice_turno
0,2023-10-19,Noite,7568,3
1,2023-10-19,Tarde,10393,2
2,2023-10-20,Manhã,27387,1
3,2023-10-20,Tarde,6808,2
4,2023-10-21,Manhã,13250,1


In [ ]:
df_desempenho_video.sort_values(by='indice_turno', inplace=True)

In [ ]:
df_desempenho_video.dtypes

data_extracao                datetime64[ns]
turno_extracao               string[python]
total_visualizacoes_turno             int64
indice_turno                          int64
dtype: object

In [ ]:
cores = ['#FFA500', '#00FF00', '#FF00FF']

In [ ]:
fig = px.bar(
        data_frame=df_desempenho_video, 
        x='data_extracao', 
        y='total_visualizacoes_turno',
        text_auto=True, 
        color='turno_extracao',
        barmode='group',
        color_discrete_sequence=cores
    )
fig.update_layout(
    title_text='Desempenho Vídeo',
    showlegend=True,
    title=dict(x=0.5, font=dict(color='white')),
    plot_bgcolor='#1F2326',
    yaxis=dict(visible=False),  
    margin=dict(l=20, r=20, t=40, b=20, pad=4),
    paper_bgcolor='#1F2326',
    xaxis=dict(title='', tickfont=dict(color='white')),
    legend=dict(font=dict(color='white')),
    xaxis_tickformat = '%d/%m/%Y',
    bargap= 0.2
)
fig.update_traces(
    textfont_color='white',
    # marker_color='#246DFB',
    textfont_size=14 ,
    textposition='outside', 
    
)


# Total Víews/likes/comentarios DIA por Canal

In [ ]:
tipos = {
    'id_canal' : 'string',
    'total_visualizacoes_turno': 'int32'
}

In [ ]:
df_views_canal = pd.read_sql_query(
    ' SELECT '
    '    id_canal, '
    '    total_visualizacoes_turno '
    '    FROM total_visualizacoes_por_semana  '
    '    WHERE data_extracao="2023-10-27"  '
    '    AND ASSUNTO = "assunto_cities_skylines" '
    '    AND total_visualizacoes_turno > 0  ',
    con=engine,
    dtype=tipos
)

df_views_canal['id_canal'] = df_views_canal['id_canal'].apply(lambda x: x.capitalize())
df_views_canal.head()

,id_canal,total_visualizacoes_turno
0,Uc-n8y8iut8b44h-iqqamq4w,4
1,Uc1mk6etfmjxr4eez7c43ztq,480
2,Uc1mk6etfmjxr4eez7c43ztq,481
3,Uc1mk6etfmjxr4eez7c43ztq,1
4,Uc1mk6etfmjxr4eez7c43ztq,342


In [ ]:
df_views_canal = df_views_canal.groupby('id_canal').sum('total_visualizacoes_turno').sort_values(by='total_visualizacoes_turno', ascending=False).reset_index()


In [ ]:
df_views_canal.head()

,id_canal,total_visualizacoes_turno
0,Ucroh1v-fymunbimrkl0y0xq,49697
1,Ucfy4ggudpeecgscnepyyvsa,18613
2,Ucayih2y5jbeosubqoqhufda,15020
3,Ucotipyf8_rgzj1lptmvadaa,13960
4,Uce9jri0yq5sm6h5qrz9fzla,10397


In [ ]:
df_views_canal.dtypes

id_canal                     string[python]
total_visualizacoes_turno             int32
dtype: object

In [ ]:
df_views_canal['id_canal'] = df_views_canal['id_canal'].astype('string')

In [ ]:
df_views_canal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id_canal                   102 non-null    string
 1   total_visualizacoes_turno  102 non-null    int32 
dtypes: int32(1), string(1)
memory usage: 1.3 KB


In [ ]:
df_views_canal.head()

,id_canal,total_visualizacoes_turno
0,Ucroh1v-fymunbimrkl0y0xq,49697
1,Ucfy4ggudpeecgscnepyyvsa,18613
2,Ucayih2y5jbeosubqoqhufda,15020
3,Ucotipyf8_rgzj1lptmvadaa,13960
4,Uce9jri0yq5sm6h5qrz9fzla,10397


In [ ]:
df_views_canal = df_views_canal.head(10)

In [ ]:
df_views_canal.head()

,id_canal,total_visualizacoes_turno
0,Ucroh1v-fymunbimrkl0y0xq,49697
1,Ucfy4ggudpeecgscnepyyvsa,18613
2,Ucayih2y5jbeosubqoqhufda,15020
3,Ucotipyf8_rgzj1lptmvadaa,13960
4,Uce9jri0yq5sm6h5qrz9fzla,10397


In [ ]:
fig = px.bar(
    df_views_canal, 
    x='total_visualizacoes_turno',
    y='id_canal', 
    orientation='h', 
    text_auto=True, 
    category_orders={'id_canal': df_views_canal['id_canal']}
)
fig.update_layout(
    title_text='TOP 10 VIEWS dia',
    showlegend=True,
    title=dict(x=0.5, font=dict(color='white')),
    plot_bgcolor='#1F2326',
    margin=dict(l=10, r=20, t=40, b=20, pad=2),
    paper_bgcolor='#1F2326',
    yaxis=dict(title='', tickfont=dict(color='white', size=16), tickmode='array', ticklen=1),
    xaxis=dict(title='', tickfont=dict(color='white'), visible=False),
    legend=dict(font=dict(color='white'), orientation='h', y=4)
)
    
fig.update_traces(
    textfont_color='white',
    marker_color='#F11A8E',
    textfont_size=18,
    textposition='outside',  
)
fig.show() 

# A duração do vídeo é importante para visualizações e interação (curtidas/comentários)?

In [ ]:
tipos = {
    'duracao_video_minutos': 'float',
    'total_comentarios_turno' : 'int32',
    'total_likes_turno': 'int32',
    'total_visualizacoes_turno': 'int32'
}

In [ ]:
df_duracao_video = pd.read_sql_query(
    ' SELECT duracao_video_minutos, '
    ' total_comentarios_turno,'
    ' total_likes_turno, '
    ' total_visualizacoes_turno'
    ' FROM total_visualizacoes_por_semana '
    ' WHERE ID_CANAL IN ("UCrOH1V-FyMunBIMrKL0y0xQ") '
    '    AND data_extracao="2023-10-27"  '
    ,
    con=engine,
    dtype=tipos
    )
df_duracao_video.head()

,duracao_video_minutos,total_comentarios_turno,total_likes_turno,total_visualizacoes_turno
0,0.00,0,50,0
1,0.00,0,50,0
2,0.00,0,50,0
3,26.88,3,165,2095
4,26.88,220,28,251


In [ ]:
fig = px.scatter(df_duracao_video, x='duracao_video_minutos', y='total_comentarios_turno', trendline='ols')
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=1)
fig.update_yaxes(title_text='TOTAL_LIKES', row=1, col=1)
fig.show()

In [ ]:
results = px.get_trendline_results(fig)

print(results.px_fit_results.iloc[0].summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     8.691
Date:                Mon, 11 Dec 2023   Prob (F-statistic):            0.00395
Time:                        21:11:09   Log-Likelihood:                -618.75
No. Observations:                 106   AIC:                             1241.
Df Residuals:                     104   BIC:                             1247.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.9472     17.567      1.306      0.1

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['DURACAO_VIDEO_MINUTOS X TOTAL_LIKES', 'DURACAO_VIDEO_MINUTOS X TOTAL_COMENTARIOS', 'DURACAO_VIDEO_MINUTOS X TOTAL_VISUALIZACOES'])
sc1 = px.scatter(df_duracao_video, x='duracao_video_minutos', y='total_likes',)
fig.add_trace(sc1['data'][0], row=1, col=1)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=1)
fig.update_yaxes(title_text='TOTAL_LIKES', row=1, col=1)

sc2 = px.scatter(df_duracao_video, x='duracao_video_minutos', y='total_comentarios')
fig.add_trace(sc2['data'][0], row=1, col=2)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=2)
fig.update_yaxes(title_text='TOTAL_COMENTARIOS', row=1, col=2)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)

sc3 = px.scatter(df_duracao_video, x='duracao_video_minutos', y='total_visualizacoes')
fig.add_trace(sc3['data'][0], row=1, col=3)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=3)
fig.update_yaxes(title_text='TOTAL_VISUALIZACOES', row=1, col=3)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)


# Desepenho dos Canais (like, comentários, vísualizações) Faça Comparação


In [ ]:
tipos = {
    'data_extracao': 'datetime64[ns]',
    'id_canal': 'string',
    'total_visualizacoes_turno' : 'int32',
    'total_likes_turno' : 'int32',
    'total_comentarios_turno' : 'int32'
}

In [ ]:
df_duracao_video = pd.read_sql_query(
    ' SELECT data_extracao, ' 
    ' id_canal, ' 
    ' total_visualizacoes_turno, ' 
    ' total_likes_turno  , ' 
    ' total_comentarios_turno '
    ' FROM total_visualizacoes_por_semana '
    ' WHERE  ID_CANAL IN ("UCrOH1V-FyMunBIMrKL0y0xQ" , "UCe9jrI0YQ5SM6h5QRZ9FZlA")  ',
    dtype=tipos,
    con=engine
    )

df_duracao_video.head()

,data_extracao,id_canal,total_visualizacoes_turno,total_likes_turno,total_comentarios_turno
0,2023-10-15,UCrOH1V-FyMunBIMrKL0y0xQ,1608,156,23
1,2023-10-15,UCrOH1V-FyMunBIMrKL0y0xQ,2954,531,23
2,2023-10-16,UCrOH1V-FyMunBIMrKL0y0xQ,1907,139,4
3,2023-10-16,UCrOH1V-FyMunBIMrKL0y0xQ,269,31,1
4,2023-10-16,UCrOH1V-FyMunBIMrKL0y0xQ,661,66,1


In [ ]:
df_duracao_video = df_duracao_video.groupby(['data_extracao', 'id_canal']) \
    .agg(
        total__visualizacoes=('total_visualizacoes_turno', 'sum'),
        total_likes=('total_likes_turno', 'sum'),
        total_comentarios=('total_comentarios_turno', 'sum')
    ).reset_index()

In [ ]:
df_duracao_video.head()

,data_extracao,id_canal,total__visualizacoes,total_likes,total_comentarios
0,2023-10-15,UCrOH1V-FyMunBIMrKL0y0xQ,9124,1374,92
1,2023-10-16,UCrOH1V-FyMunBIMrKL0y0xQ,21502,3506,122
2,2023-10-19,UCe9jrI0YQ5SM6h5QRZ9FZlA,20649,4817,336
3,2023-10-19,UCrOH1V-FyMunBIMrKL0y0xQ,133688,21350,1464
4,2023-10-20,UCe9jrI0YQ5SM6h5QRZ9FZlA,28906,3486,272


In [ ]:
fig = px.line(df_duracao_video, x='data_extracao', y='total__visualizacoes', color='id_canal')
fig.update_layout(
    title_text='Comparação Visualizacao',
    showlegend=True,
    title=dict(x=0.5, font=dict(color='white')),
    plot_bgcolor='#1F2326',
    yaxis=dict(visible=False),  
    margin=dict(l=20, r=20, t=40, b=20, pad=6),
    paper_bgcolor='#1F2326',
    xaxis=dict(title='', tickfont=dict(color='white'), showgrid=False),
    legend=dict(font=dict(color='white')),
    xaxis_tickformat = '%d/%m/%Y',
    bargap=0.09

)
fig.update_traces(
    textfont_color='white',
    textfont_size=16 ,
     
)
fig.show()

# Wordcloud TAGS

In [ ]:
df_wordcloud_tags = pd.read_sql_query(
    ' SELECT tags '
    ' FROM total_visualizacoes_por_semana '
    ' WHERE assunto = "assunto_cities_skylines" ',
    con=engine
)
df_wordcloud_tags.head()

,tags
0,[]
1,"[""cities skylines"",""cities skylines gameplay"",..."
2,"[""xbae8vvXZp1LBYAKst0k"",""Machinimas"",""GAC"",""Ga..."
3,"[""euro truck 2"",""lucas gameplays brasil"",""mapa..."
4,"[""review"",""gameplay"",""jogo pc"",""jogos pc"",""pc ..."


In [ ]:
df_wordcloud_tags_copy = df_wordcloud_tags.copy()

In [ ]:
df_wordcloud_tags_copy.drop_duplicates(inplace=True)

In [ ]:
df_wordcloud_tags_copy.head()

,tags
0,[]
1,"[""cities skylines"",""cities skylines gameplay"",..."
2,"[""xbae8vvXZp1LBYAKst0k"",""Machinimas"",""GAC"",""Ga..."
3,"[""euro truck 2"",""lucas gameplays brasil"",""mapa..."
4,"[""review"",""gameplay"",""jogo pc"",""jogos pc"",""pc ..."


In [ ]:
teste = ["cities skylines","cities skylines gameplay"]

def partir_lista(teste):
     return ','.join(str(item) for item in teste)

partir_lista(teste=teste)

'cities skylines,cities skylines gameplay'

In [ ]:
def converter_para_lista(texto):
    texto = texto.replace('[', '').replace(']', '').replace('"', '')
    lista = texto.split(', ')
    return lista


In [ ]:
df_wordcloud_tags_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108 entries, 0 to 1446
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tags    108 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [ ]:
df_wordcloud_tags_copy['tags'] = df_wordcloud_tags_copy['tags'].apply(converter_para_lista)

In [ ]:
df_wordcloud_tags_copy.head()

,tags
0,[]
1,"[cities skylines,cities skylines gameplay,citi..."
2,"[xbae8vvXZp1LBYAKst0k,Machinimas,GAC,Gameplay,..."
3,"[euro truck 2,lucas gameplays brasil,mapa eaa,..."
4,"[review,gameplay,jogo pc,jogos pc,pc game,joga..."


In [ ]:
df_wordcloud_tags_copy['tags_str'] = df_wordcloud_tags_copy['tags'].apply(partir_lista)
df_wordcloud_tags_copy.head()

,tags,tags_str
0,[],
1,"[cities skylines,cities skylines gameplay,citi...","cities skylines,cities skylines gameplay,citie..."
2,"[xbae8vvXZp1LBYAKst0k,Machinimas,GAC,Gameplay,...","xbae8vvXZp1LBYAKst0k,Machinimas,GAC,Gameplay,P..."
3,"[euro truck 2,lucas gameplays brasil,mapa eaa,...","euro truck 2,lucas gameplays brasil,mapa eaa,m..."
4,"[review,gameplay,jogo pc,jogos pc,pc game,joga...","review,gameplay,jogo pc,jogos pc,pc game,jogan..."


- Média likes, comentarios like / visualizacoes ou likes / visualizacoes

# Métricas especificas TRENDS

# Categrias Populares por data de extração

In [ ]:
tipos = {
    'data_extracao': 'datetime64[ns]',
    'nm_canal': 'string',
    'titulo_video': 'string',
    'id_categoria': 'string',
    'total_visualizacoes_turno': 'int32',
    'total_likes_turno' : 'float32',
    'total_comentarios_turno': 'float32',
    'turno_extracao' : 'string'
}

In [ ]:
df_categoria_populares = pd.read_sql_query(
    ' SELECT data_extracao, ' 
    ' nm_canal, '
    ' titulo_video, '
    ' id_categoria, ' 
    ' total_visualizacoes_turno, '  
    ' total_likes_turno, '
    ' total_comentarios_turno, '
    ' turno_extracao '
    '  FROM trends_youtube '
    ' where data_extracao = "2023-10-15" ',
    con=engine,
    dtype=tipos
    )
df_categoria_populares.head()

,data_extracao,nm_canal,titulo_video,id_categoria,total_visualizacoes_turno,total_likes_turno,total_comentarios_turno,turno_extracao
0,2023-10-15,東映アニメーション公式YouTubeチャンネル,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,1,70675,1693.0,107.0,Noite
1,2023-10-15,東映アニメーション公式YouTubeチャンネル,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,1,4070343,170259.0,17651.0,Tarde
2,2023-10-15,Paulinho e Toquinho Family,INCRÍVEL PISCINA de BOLINHAS DENTRO do CARRO,1,1457,13.0,114.0,Noite
3,2023-10-15,Paulinho e Toquinho Family,INCRÍVEL PISCINA de BOLINHAS DENTRO do CARRO,1,268434,3671.0,114.0,Tarde
4,2023-10-15,A24,The Iron Claw | Official Trailer HD | A24,1,133788,1996.0,92.0,Noite


In [ ]:
df_categoria_populares.memory_usage(deep=True)

Index                          128
data_extracao                 3464
nm_canal                     30932
titulo_video                 60367
id_categoria                 25537
total_visualizacoes_turno     1732
total_likes_turno             1732
total_comentarios_turno       1732
turno_extracao               26846
dtype: int64

In [ ]:
df_categoria_populares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   data_extracao              433 non-null    datetime64[ns]
 1   nm_canal                   433 non-null    string        
 2   titulo_video               433 non-null    string        
 3   id_categoria               433 non-null    string        
 4   total_visualizacoes_turno  433 non-null    int32         
 5   total_likes_turno          423 non-null    float32       
 6   total_comentarios_turno    416 non-null    float32       
 7   turno_extracao             433 non-null    string        
dtypes: datetime64[ns](1), float32(2), int32(1), string(4)
memory usage: 22.1 KB


In [ ]:
df_categoria_populares.groupby(['data_extracao', 'id_categoria']) \
    .agg(
        total_visualizacoes=('total_visualizacoes_turno', 'sum'),
        total_likes = ('total_likes_turno', 'sum'),
        total_comentarios_dia=('total_comentarios_turno', 'sum')
    ).reset_index()

,data_extracao,id_categoria,total_visualizacoes,total_likes,total_comentarios_dia
0,2023-10-15,1,10646952,487262.0,36906.0
1,2023-10-15,10,188470120,8054410.0,505272.0
2,2023-10-15,17,25971018,959911.0,61992.0
3,2023-10-15,20,29314189,1754983.0,117554.0
4,2023-10-15,22,6772581,620345.0,51419.0
5,2023-10-15,23,1361139,68949.0,2195.0
6,2023-10-15,24,285267745,11749035.0,575284.0
7,2023-10-15,25,11520764,253153.0,13780.0
8,2023-10-15,26,49953,9076.0,690.0
9,2023-10-15,27,5215689,263761.0,12350.0


- Vídeo populares (likes, comentarios e visualizações)

In [ ]:
df_categoria_populares.head()

,data_extracao,nm_canal,titulo_video,id_categoria,total_visualizacoes_turno,total_likes_turno,total_comentarios_turno,turno_extracao
0,2023-10-15,東映アニメーション公式YouTubeチャンネル,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,1,70675,1693.0,107.0,Noite
1,2023-10-15,東映アニメーション公式YouTubeチャンネル,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,1,4070343,170259.0,17651.0,Tarde
2,2023-10-15,Paulinho e Toquinho Family,INCRÍVEL PISCINA de BOLINHAS DENTRO do CARRO,1,1457,13.0,114.0,Noite
3,2023-10-15,Paulinho e Toquinho Family,INCRÍVEL PISCINA de BOLINHAS DENTRO do CARRO,1,268434,3671.0,114.0,Tarde
4,2023-10-15,A24,The Iron Claw | Official Trailer HD | A24,1,133788,1996.0,92.0,Noite


- Canais populares (likes, comentarios e visualizações)

In [ ]:
df_categoria_populares.groupby(['data_extracao', 'nm_canal']) \
    .agg(
        total_visualizacoes=('total_visualizacoes_turno', 'sum'),
        total_likes = ('total_likes_turno', 'sum'),
        total_comentarios_dia=('total_comentarios_turno', 'sum')
    ).reset_index()

,data_extracao,nm_canal,total_visualizacoes,total_likes,total_comentarios_dia
0,2023-10-15,(G)I-DLE (여자)아이들 (Official YouTube Channel),8627209,476803.0,19048.0
1,2023-10-15,30PRAUM,932037,116511.0,9296.0
2,2023-10-15,5incominutos,165650,27148.0,1994.0
3,2023-10-15,A Fazenda,1190662,25723.0,0.0
4,2023-10-15,A24,4279413,78174.0,5415.0
...,...,...,...,...,...
183,2023-10-15,ge,3921474,109948.0,9405.0
184,2023-10-15,k a m a i t a c h i,113624,28687.0,2104.0
185,2023-10-15,starshipTV,29838674,780081.0,45617.0
186,2023-10-15,viniccius13,1607585,259444.0,10361.0


- Desempenho da categoria por dia

In [ ]:
tipos = {
    'data_extracao': 'datetime64[ns]',
    'nm_canal': 'string',
    'titulo_video': 'string',
    'id_categoria': 'integer',
    'total_visualizacoes_turno': 'int32',
    'total_likes_turno' : 'float32',
    'total_comentarios_turno': 'float32',
    'turno_extracao' : 'string'
}

In [ ]:
df_desempenho_categoria_dia = pd.read_sql_query(
    ' SELECT data_extracao, ' 
    ' nm_canal, '
    ' titulo_video, '
    ' id_categoria, ' 
    ' total_visualizacoes_turno, '  
    ' total_likes_turno, '
    ' total_comentarios_turno, '
    ' turno_extracao ' 
    ' FROM trends_youtube '
    ' where id_categoria = 20  ',
    con=engine,
    dtype=tipos
    )
df_desempenho_categoria_dia.head()

,data_extracao,nm_canal,titulo_video,id_categoria,total_visualizacoes_turno,total_likes_turno,total_comentarios_turno,turno_extracao
0,2023-10-15,PlayStation,PlayStation 5 - Same Immersive Power. New Slim...,20,9680,106.0,6.0,Noite
1,2023-10-15,PlayStation,PlayStation 5 - Same Immersive Power. New Slim...,20,1471570,38517.0,3771.0,Tarde
2,2023-10-15,Flakes Power,URGENTE!!! FORTNITE,20,1054,42.0,3.0,Noite
3,2023-10-15,Flakes Power,URGENTE!!! FORTNITE,20,194833,10562.0,488.0,Tarde
4,2023-10-15,Ana Toys games,"COMO ACHAR TODOS OS DOCES EXTREME, HARD E INSA...",20,3204,75.0,2.0,Noite


In [ ]:
df_desempenho_categoria_dia.groupby('data_extracao').agg(
    total_visualizacoes=('total_visualizacoes_turno', 'sum'),
    total_likes=('total_likes_turno', 'sum'),
    total_comentarios=('total_comentarios_turno', 'sum')
).reset_index()

,data_extracao,total_visualizacoes,total_likes,total_comentarios
0,2023-10-15,29314189,1754983.0,117554.0
1,2023-10-16,8864341,495852.0,48893.0
2,2023-10-19,8804495,430599.0,51034.0
3,2023-10-20,3520689,89122.0,10135.0
4,2023-10-21,6227678,257911.0,18309.0
5,2023-10-22,8052190,428650.0,43622.0
6,2023-10-23,8572491,713170.0,54658.0
7,2023-10-24,9262972,450165.0,41689.0
8,2023-10-25,4755872,183938.0,45592.0
9,2023-10-26,6484170,255852.0,60162.0


- Top 10 vídeo em alta por categoria

In [ ]:
tipos = {
    'data_extracao': 'datetime64[ns]',
    'turno_extracao' : 'string',
    'nm_canal': 'string',
    'titulo_video': 'string',
    'id_categoria': 'string',
    'total_visualizacoes_turno': 'int32',
    'total_likes_turno' : 'float32',
    'total_comentarios_turno': 'float32'
}

In [ ]:
df_top_10 = pd.read_sql_query(
    ' SELECT  ' 
    ' data_extracao, '
    ' turno_extracao,'
    ' nm_canal, '
    ' titulo_video, '
    ' id_categoria, ' 
    ' total_visualizacoes_turno, '  
    ' total_likes_turno, '
    ' total_comentarios_turno '
    '  FROM trends_youtube '
    ' where id_categoria = 20 '
    ' AND data_extracao = "2023-10-26" ',
    con=engine,
    dtype=tipos
    )
df_top_10.head(10)

,data_extracao,turno_extracao,nm_canal,titulo_video,id_categoria,total_visualizacoes_turno,total_likes_turno,total_comentarios_turno
0,2023-10-26,Manhã,Apex Legends,Apex Legends: Ignite Launch Trailer,20,72260,1642.0,180.0
1,2023-10-26,Noite,Apex Legends,Apex Legends: Ignite Launch Trailer,20,24690,537.0,15.0
2,2023-10-26,Tarde,Apex Legends,Apex Legends: Ignite Launch Trailer,20,50593,1069.0,25.0
3,2023-10-26,Noite,Apex Legends,Apex Legends: Ignite Gameplay Trailer,20,840241,41044.0,3831.0
4,2023-10-26,Manhã,League of Legends,"HEARTSTEEL - PARANOIA ft. BAEKHYUN, tobi lou, ...",20,988062,18454.0,982.0
5,2023-10-26,Noite,League of Legends,"HEARTSTEEL - PARANOIA ft. BAEKHYUN, tobi lou, ...",20,405627,5923.0,399.0
6,2023-10-26,Tarde,League of Legends,"HEARTSTEEL - PARANOIA ft. BAEKHYUN, tobi lou, ...",20,542301,9670.0,386.0
7,2023-10-26,Manhã,League of Legends,HEARTSTEEL - PARANOIA Music Video Teaser | Lea...,20,7181,369.0,2.0
8,2023-10-26,Noite,League of Legends,HEARTSTEEL - PARANOIA Music Video Teaser | Lea...,20,3371,93.0,2150.0
9,2023-10-26,Tarde,League of Legends,HEARTSTEEL - PARANOIA Music Video Teaser | Lea...,20,5464,135.0,1.0


In [ ]:
df_top_10.groupby(['data_extracao', 'nm_canal', 'titulo_video']).agg(
    total_visualizacoes=('total_visualizacoes_turno', 'sum'),
    total_likes=('total_likes_turno', 'sum'),
    total_comentarios=('total_comentarios_turno', 'sum')
).reset_index()

,data_extracao,nm_canal,titulo_video,total_visualizacoes,total_likes,total_comentarios
0,2023-10-26,ARK: Survival Ascended,ARK: Survival Ascended Launch Trailer,1322140,66120.0,7019.0
1,2023-10-26,Apex Legends,Apex Legends: Ignite Gameplay Trailer,840241,41044.0,3831.0
2,2023-10-26,Apex Legends,Apex Legends: Ignite Launch Trailer,147543,3248.0,220.0
3,2023-10-26,BREN0RJ,CONSEGUI O NOVO ESTILO DE LUTA DO BLOX FRUITS ...,8120,443.0,147.0
4,2023-10-26,BREN0RJ,UPEI DO 1 AO 700 COM A NOVA FRUTA MAMUTE DO BL...,31674,1356.0,26.0
5,2023-10-26,BRKsEDU,"SPIDER-MAN 2 - O Início de Gameplay, em Portug...",28297,1007.0,7.0
6,2023-10-26,Bern,Eu EXPULSEI o GELEIA da CREATIVE SQUAD,3567,113.0,417.0
7,2023-10-26,Brawl Stars,Brawl Stars Animation - The Bizarre Circus Spe...,75088,1358.0,72.0
8,2023-10-26,Brawl Stars,"Brawl Stars: Brawl Talk - MEGA PIG, Spider Bra...",123645,1671.0,164.0
9,2023-10-26,ChratosGameplay,PEQUENA FORTUNA EM SERVIÇOS PÚBLICOS BÁSICOS 🏙...,2813,214.0,372.0


- Encontre o máximo min e média baseada por palavra chave

- Total de Visualizacoes, comentários e likes por assunto x data

In [59]:
base_desempenho = pd.read_parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_cities_skylines/total_visualizacoes_por_semana/total_visualizacoes_por_semana.parquet')
base_desempenho.columns


Index(['INDICE_TURNO_EXTRACAO', 'NM_CANAL', 'ID_VIDEO', 'TITULO_VIDEO',
       'TOTAL_CARACTERE_VIDEO', 'TAGS', 'DURACAO_VIDEO_MINUTOS', 'TOTAL_TAGS',
       'TOTAL_VISUALIZACOES', 'TOTAL_COMENTARIOS', 'TOTAL_LIKES',
       'TOTAL_VISUALIZACOES_TURNO', 'TOTAL_COMENTARIOS_TURNO',
       'TOTAL_LIKES_TURNO', 'data_extracao', 'TURNO_EXTRACAO', 'ID_CANAL'],
      dtype='object')

In [60]:
base_desempenho.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   INDICE_TURNO_EXTRACAO      3951 non-null   int32   
 1   NM_CANAL                   3951 non-null   object  
 2   ID_VIDEO                   3951 non-null   object  
 3   TITULO_VIDEO               3951 non-null   object  
 4   TOTAL_CARACTERE_VIDEO      3951 non-null   int32   
 5   TAGS                       2916 non-null   object  
 6   DURACAO_VIDEO_MINUTOS      3951 non-null   float32 
 7   TOTAL_TAGS                 3951 non-null   int32   
 8   TOTAL_VISUALIZACOES        3951 non-null   int32   
 9   TOTAL_COMENTARIOS          3951 non-null   int32   
 10  TOTAL_LIKES                3896 non-null   float64 
 11  TOTAL_VISUALIZACOES_TURNO  3951 non-null   int32   
 12  TOTAL_COMENTARIOS_TURNO    3951 non-null   int32   
 13  TOTAL_LIKES_TURNO          3896 n

In [61]:
def indice_semana( dia: str) -> int | None:
        dias_semana = {
            'Domingo': 1,
            'Segunda-feira': 2,
            'Terça-feira': 3,
            'Quarta-feira': 4,
            'Quinta-feira': 5,
            'Sexta-feira': 6,
            'Sábado': 7
        }

        return dias_semana.get(dia)

In [62]:
base_desempenho = base_desempenho[[ 'TOTAL_VISUALIZACOES_TURNO', 'data_extracao']]

In [63]:
base_desempenho.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   TOTAL_VISUALIZACOES_TURNO  3951 non-null   int32   
 1   data_extracao              3951 non-null   category
dtypes: category(1), int32(1)
memory usage: 19.8 KB


In [64]:
base_desempenho.head()

,TOTAL_VISUALIZACOES_TURNO,data_extracao
0,97,2023-10-15
1,1,2023-10-15
2,4,2023-10-15
3,3,2023-10-15
4,100,2023-10-15


In [65]:
base_desempenho = base_desempenho.groupby('data_extracao', observed=False).agg(
    TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES_TURNO', 'sum')
).reset_index()
base_desempenho.head()

,data_extracao,TOTAL_VISUALIZACOES
0,2023-10-15,6454
1,2023-10-16,14432
2,2023-10-19,105004
3,2023-10-20,104625
4,2023-10-21,156661
